In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import pandas as pd

In [43]:
# URL of page to be scraped
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

Get the latest news

In [44]:
# Retrieve page with the requests module
response = requests.get(url)
print(response)

<Response [200]>


In [45]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [46]:
# results are returned as an iterable list
results = soup.find_all('div', class_="slide")


In [48]:
#Get the latest news article (title and text)
Latest_news = soup.find('div', class_="content_title").text.strip()
Latest_para = soup.find('div', class_="rollover_description_inner").text.strip()
Latest_para

"Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover."

Get the featured image

In [8]:
#Activate chromedriver
!which chromedriver

In [40]:
#URL of page to be scraped
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
base_url ="https://www.jpl.nasa.gov"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

article = soup.find("article")
f_img_url = base_url + article.a['data-fancybox-href']
f_img_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16606_ip.jpg'

Get the latest Mars Tweet

In [11]:
url = "https://twitter.com/marswxreport?lang=en"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
    
tweet = soup.find_all("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")[0].text
tweet

'We won’t be hearing from @MarsCuriosity or @NASAInSight for the next 2 weeks during Mars solar conjunction. Read more about why Mars missions go silent every 2 years: https://www.wral.com/mars-spacecraft-go-quiet-during-solar-conjunction/18595551/\xa0…pic.twitter.com/fWruE2v151'

Get facts about Mars

In [12]:
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)

df = tables[1]
df.columns = ['description', 'value']
df.set_index('description', inplace=True)

html_table = df.to_html()
html_table = html_table.replace('\n', '')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

Get Mars hemispheres

In [19]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

results = soup.find_all("div", class_="description")

hemispheres = []
for result in results:
    hemispheres.append(result.h3.text)
hemispheres
    

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [32]:
#Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres
#You will need to click each of the links to the hemispheres in order to find the image url to the full 
#resolution image.
#Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the 
#hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
#Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one 
#dictionary for each hemisphere.

executable_path = {'executable_path': '/Users/chrismiller/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

#create the empty list
hemisphere_list = []

#set up the search adn dict
for hemisphere in hemispheres:
    browser.click_link_by_partial_text(f'{hemisphere}')
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    image_link = soup.find('img', class_='wide-image')['src']
    image_url = 'https://astrogeology.usgs.gov' + image_link
    hemi_dict = {}
    
    #fill the dict
    hemi_dict["title"] = f'{hemisphere}'
    hemi_dict["img_url"] = image_url
    hemisphere_list.append(hemi_dict)

    # Return to the original page
    browser.visit(url)
        
browser.quit()
hemisphere_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]